In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import numpy as np
from datetime import datetime

from XDCCA.data.mnist import MNISTData
from XDCCA.experiments.mnist import MNISTConvDeepCCAExperiment

In [ ]:
# Load synthetic data
mnist_dataprovider = MNISTData.generate(25000, num_boxes=0, max_width=10, flatten=False)

# Create root dir for all following experiments
root_dir = ''+datetime.now().strftime("%d_%m_%Y_%H_%M_%S")
os.mkdir(root_dir)

# Save dataset
mnist_dataprovider.save(root_dir)

---

# DCCAE

In [ ]:
for n_filters in [15]:
    for l2 in [1e-3, 1e-2, 1e-1]:
        for _ in range(3):
            exp = MNISTConvDeepCCAExperiment(
                log_dir=os.path.join(root_dir, 'dcca'), 
                encoder_config_v1=[
                    dict(l_type='conv', n_filters=5, k_size=7, activation=None),
                    dict(l_type='maxpool', pool_size=4),
                    dict(l_type='conv', n_filters=10, k_size=5, activation=None),
                    dict(l_type='maxpool', pool_size=7),
                    dict(l_type='conv', n_filters=n_filters, k_size=3, activation=None),
                ],
                encoder_config_v2=[
                    dict(l_type='conv', n_filters=5, k_size=7, activation=None),
                    dict(l_type='maxpool', pool_size=4),
                    dict(l_type='conv', n_filters=10, k_size=5, activation=None),
                    dict(l_type='maxpool', pool_size=7),
                    dict(l_type='conv', n_filters=n_filters, k_size=3, activation=None),
                ],
                dataprovider=mnist_dataprovider,
                shared_dim=15,
                lambda_l2=l2,
                cca_reg=1e-4,
                eval_epochs=10,
                val_default_value=0.0,
            )

            exp.train_multiple_epochs(300)

            exp.save()